In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
path_to_files = "stock_data/"
# path_to_sentiments1 = "stock_news_sentiment_scores/2011-2024/stock_news_sentiment_analysis_results_"
path_to_sentiments = "stock_news_sentiment_scores/2011-2024/stock_news_sentiment_analysis_results_"

stock_symbols = ['GOOG', 'MSFT', 'NVDA','AMZN','AAPL']

k = 0
print(stock_symbols[k])

GOOG


In [3]:
stock_sentiment = pd.read_csv(path_to_sentiments+stock_symbols[k]+".csv")
stock_price = pd.read_csv(path_to_files+stock_symbols[k]+"/stock_price.csv")

In [4]:
print(stock_sentiment.keys())
print(stock_price.keys())

Index(['Date', 'Ticker', 'weighted positive sentiment score',
       'weighted negative sentiment score', 'weighted neutral sentiment score',
       'weighted compound sentiment score'],
      dtype='object')
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits'],
      dtype='object')


In [5]:
stock_sentiment.sample(10)

,Date,Ticker,weighted positive sentiment score,weighted negative sentiment score,weighted neutral sentiment score,weighted compound sentiment score
198,2020-01-07,GOOG,0.379000,0.000000,0.969117,0.771700
409,2020-09-29,GOOG,0.151402,0.215994,0.804893,-0.238550
1134,2023-03-28,GOOG,0.237323,0.161298,0.871940,0.259833
670,2021-09-15,GOOG,0.238931,0.213614,0.844597,0.432279
274,2020-04-02,GOOG,0.237318,0.216533,0.843700,0.045000
684,2021-09-30,GOOG,0.188469,0.231812,0.678227,-0.106067
1269,2023-09-06,GOOG,0.369491,0.238663,0.787118,0.166001
515,2021-02-10,GOOG,0.328296,0.109393,0.852713,0.600120
702,2021-10-19,GOOG,0.228173,0.111000,0.862312,0.420519
651,2021-08-23,GOOG,0.000000,0.145000,0.956576,-0.296000


In [20]:
# plt.plot(stock_sentiment["Date"], stock_sentiment["Weighted compound sentiment score"])

def categorize_change(x):
    if pd.isnull(x):
        return None  # For the first day
    elif x > 0:
        return 1
    else:
        return 0

In [ ]:
# def compute_stock_features(stock_symbols, start_date_, end_date_, sma_interval):
#     # Initialize dictionaries
#     df_sentiment_scores_dict = {}
#     df_stock_data_dict = {}
#     sentiment_scores = {}
#     stock_data = {}
#     stock_diff_close_open = {}

#     start_date = pd.to_datetime('2012-01-01').date()
#     end_date = pd.to_datetime('2022-01-01').date()

#     for symbol in stock_symbols:
#         # Load sentiment and stock data
#         df_sentiment_scores_dict[symbol] = pd.read_csv(path_to_sentiments+symbol+".csv")
#         df_stock_data_dict[symbol] = pd.read_csv(path_to_files+symbol+"/stock_price.csv")

#     # Convert date columns to datetime and extract date part only as `datetime.date`
#     df_sentiment_scores_dict[symbol]['Date'] = pd.to_datetime(df_sentiment_scores_dict[symbol]['Date'], utc = True).dt.date
#     df_stock_data_dict[symbol]['Date'] = pd.to_datetime(df_stock_data_dict[symbol]['Date'], utc= True).dt.date
    
#     # Filter sentiment scores based on date range
#     mask_sentiment = (df_sentiment_scores_dict[symbol]['Date'] >= start_date) & (df_sentiment_scores_dict[symbol]['Date'] <= end_date)
#     sentiment_filtered = df_sentiment_scores_dict[symbol][mask_sentiment]

#     # Filter stock data based on date range and intersecting dates
#     stock_dates = df_stock_data_dict[symbol]['Date']
#     mask_stock = (stock_dates >= start_date) & (stock_dates <= end_date) & (stock_dates.isin(sentiment_filtered['Date']))
#     stock_filtered = df_stock_data_dict[symbol][mask_stock]

#     # Align both DataFrames by keeping only common dates
#     common_dates = sentiment_filtered['Date'][sentiment_filtered['Date'].isin(stock_filtered['Date'])]
#     sentiment_aligned = sentiment_filtered[sentiment_filtered['Date'].isin(common_dates)]
#     stock_aligned = stock_filtered[stock_filtered['Date'].isin(common_dates)]

#     # Put all data together 
#     full_stock_data = pd.concat([stock_aligned, sentiment_aligned], axis = 1)

#     full_stock_data['SMA'] = full_stock_data['Close'].rolling(window=7).mean()
#     full_stock_data['Price Difference'] = full_stock_data['Close'].diff().apply(categorize_change)
#     # print(full_stock_data.keys())
#     print(full_stock_data.shape, symbol)
    

#     stock_data[symbol] = full_stock_data
#     X_features = full_stock_data.keys()    

In [31]:
# Initialize dictionaries
df_sentiment_scores_dict = {}
df_stock_data_dict = {}
sentiment_scores = {}
stock_data = {}
stock_diff_close_open = {}

start_date = pd.to_datetime('2012-01-01').date()
end_date = pd.to_datetime('2022-01-01').date()

for symbol in stock_symbols:
    # Load sentiment and stock data
    df_sentiment_scores_dict[symbol] = pd.read_csv(path_to_sentiments+symbol+".csv")
    df_stock_data_dict[symbol] = pd.read_csv(path_to_files+symbol+"/stock_price.csv")

    # Convert date columns to datetime and extract date part only as `datetime.date`
    df_sentiment_scores_dict[symbol]['Date'] = pd.to_datetime(df_sentiment_scores_dict[symbol]['Date'], utc = True).dt.date
    df_stock_data_dict[symbol]['Date'] = pd.to_datetime(df_stock_data_dict[symbol]['Date'], utc= True).dt.date
    
    # Filter sentiment scores based on date range
    mask_sentiment = (df_sentiment_scores_dict[symbol]['Date'] >= start_date) & (df_sentiment_scores_dict[symbol]['Date'] <= end_date)
    sentiment_filtered = df_sentiment_scores_dict[symbol][mask_sentiment]

    # Filter stock data based on date range and intersecting dates
    stock_dates = df_stock_data_dict[symbol]['Date']
    mask_stock = (stock_dates >= start_date) & (stock_dates <= end_date) & (stock_dates.isin(sentiment_filtered['Date']))
    stock_filtered = df_stock_data_dict[symbol][mask_stock]

    # Align both DataFrames by keeping only common dates
    common_dates = sentiment_filtered['Date'][sentiment_filtered['Date'].isin(stock_filtered['Date'])]
    sentiment_aligned = sentiment_filtered[sentiment_filtered['Date'].isin(common_dates)]
    stock_aligned = stock_filtered[stock_filtered['Date'].isin(common_dates)]
    print(stock_aligned.shape, sentiment_aligned.shape)

    # Put all data together 
    full_stock_data = pd.concat([stock_aligned, sentiment_aligned[2:]], axis = 1).reindex(stock_aligned.index)

    full_stock_data['SMA'] = full_stock_data['Close'].rolling(window=7).mean()
    full_stock_data['Price Difference'] = full_stock_data['Close'].diff().apply(categorize_change)
    # print(full_stock_data.keys())
    print(full_stock_data.shape, symbol)
    

    stock_data[symbol] = full_stock_data
    X_features = full_stock_data.keys()

print(X_features)
stock_data['GOOG'].sample(10)

(642, 8) (642, 6)
(642, 16) GOOG
(666, 8) (666, 6)
(666, 16) MSFT
(560, 8) (560, 6)
(560, 16) NVDA
(818, 8) (818, 6)
(818, 16) AMZN
(863, 8) (863, 6)
(863, 16) AAPL
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'Date', 'Ticker', 'weighted positive sentiment score',
       'weighted negative sentiment score', 'weighted neutral sentiment score',
       'weighted compound sentiment score', 'SMA', 'Price Difference'],
      dtype='object')


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date,Ticker,weighted positive sentiment score,weighted negative sentiment score,weighted neutral sentiment score,weighted compound sentiment score,SMA,Price Difference
2710,2021-10-08,139.563264,139.973253,139.087778,139.712891,18924000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,136.274693,1.0
2364,2020-05-27,70.688900,70.912845,69.394076,70.718323,33716000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,69.915619,1.0
2610,2021-05-18,116.559048,116.870486,114.875880,114.889351,17302000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,115.070980,-1.0
1949,2018-10-01,59.847526,60.346800,59.369201,59.619087,27152000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,60.886332,-1.0
2273,2020-01-15,71.335307,71.893187,71.335307,71.783707,25654000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,70.943847,1.0
2728,2021-11-03,145.916652,146.550608,144.698147,146.430389,17886000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,144.919246,1.0
2307,2020-03-05,67.344618,67.779052,65.095137,65.790436,51226000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,67.595856,-1.0
1953,2018-10-05,58.231991,58.531255,57.115733,57.725735,23686000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,60.543246,-1.0
2675,2021-08-19,135.135623,137.115258,135.024400,136.578079,18296000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,137.403277,1.0
1378,2016-06-24,33.675795,34.385554,33.590007,33.678291,88980000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,35.689780,-1.0


In [28]:
print(stock_data['GOOG'].shape)
print(stock_data.keys())

columns = stock_data['GOOG'].keys()
print(columns)


stock_data['MSFT'].sample(10)

(642, 16)
dict_keys(['GOOG', 'MSFT', 'NVDA', 'AMZN', 'AAPL'])
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'Date', 'Ticker', 'weighted positive sentiment score',
       'weighted negative sentiment score', 'weighted neutral sentiment score',
       'weighted compound sentiment score', 'SMA', 'Price Difference'],
      dtype='object')


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date,Ticker,weighted positive sentiment score,weighted negative sentiment score,weighted neutral sentiment score,weighted compound sentiment score,SMA,Price Difference
1275,2016-01-28,45.835714,46.145055,45.296574,46.012482,62513800.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,44.696896,1.0
2483,2020-11-12,209.830014,211.665453,207.173449,208.120148,21593900.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,206.099806,-1.0
639,2013-07-19,26.741663,26.964508,25.602666,25.916302,248428500.0,0.0,0.0,2021-06-18,MSFT,0.0,0.0,1.0,0.0,28.131803,-1.0
528,2013-02-08,22.232328,22.524964,22.199812,22.394903,33318500.0,0.0,0.0,2021-01-06,MSFT,0.0,0.0,1.0,0.0,22.461241,1.0
2353,2020-05-11,176.010370,180.200407,175.722076,179.460434,30892700.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,174.644390,1.0
1878,2018-06-20,94.960167,96.037444,94.725975,95.428551,26180800.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,92.579142,1.0
2530,2021-01-22,219.938982,222.834961,218.699236,218.844513,30172700.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,212.620871,1.0
1276,2016-01-29,48.372294,48.690475,47.727095,48.690475,83611700.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,45.834081,1.0
2566,2021-03-16,229.376771,233.046332,229.046708,230.764999,28092200.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,227.300683,1.0
1743,2017-12-05,75.513125,76.757137,75.178921,75.745216,26152300.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,75.412336,1.0


In [32]:


sns.pairplot(data = stock_data['GOOG'],
                x_vars = ['Close', 'Volume' ],
                y_vars = ['Close', 'Volume'],
                plot_kws = {'alpha':.6})

plt.show()

ValueError: 2

In [ ]:
plt.scatter(stock_data['GOOG']['Volume'], stock_data['GOOG']['Close'])